# Day 03: Polygons

For this category, I wanted to investigate building footprints using OpenStreetMap/Overpass.

There are a variety of data sources I will use throughout these exercises, including:
* [Explorer Basemap](https://visibleearth.nasa.gov/images/147190/explorer-base-map): Joshua Stevens, NASA Earth Observatory
* [Comprehensive Global Administrative Zones (CGAZ)](https://www.geoboundaries.org/): geoBoundaries 4.0, William & Mary geoLab
* [Marine polys and lakes](https://www.naturalearthdata.com/): Natural Earth
* [World Cities](https://simplemaps.com/data/world-cities): Simple Maps

Data sources specific to a single day will be linked in the notebook where they appear.

## Configuration
Dependency imports and utility functions are in a separate `plotter.py` file in order to keep these notebooks clean.

In [1]:
import os
from geopy.geocoders import Nominatim
import overpass
from shapely.geometry import shape
import matplotlib.pyplot as plt 
import geopandas as gpd
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
# Desired styling for matplotlib
from matplotlib import cycler
colors = cycler('color',["44aa98","ab4498","332389","86ccec","ddcc76","cd6477","882255", "117732"])
plt.rcParams['figure.figsize'] = [6,4]
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['text.color'] = '212121'
plt.rcParams['xtick.color'] = '212121'
plt.rcParams['ytick.color'] = '212121'
plt.rcParams['font.family'] = 'sans serif'
plt.rcParams['axes.facecolor'] = 'None'
plt.rcParams['axes.edgecolor'] = 'dimgray'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.color'] = 'lightgray'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['xtick.labelsize'] = 'x-small'
plt.rcParams['ytick.labelsize'] = 'x-small'
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.framealpha'] = 0.8
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'None'
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['axes.labelsize'] = 'small'
plt.rcParams['savefig.facecolor'] = 'None'
plt.rcParams['savefig.edgecolor'] = 'None'
plt.rc('axes', prop_cycle=colors)

# Query OSM
I want to find a generic way I can query building footprints from OSM for a city of interest. 

First, I will need to programmatically determine the coordinates of the city of interest. I want to be able to simply type in a string with the city name, so I will use the `geocoders` package. To make use of OSM data and to use a free provider without creating any acount, I will use **Nominatim**.

Second, I will want to query building footprints using the Overpass API for OpenStreetMap. I will have to assume a certain radius around the city coordinates from step one.

Last, I will plot the features!

In [3]:
# Choose a city of interest
# NOTE: Sometimes this retrieves a region boundary rather than a city location. 
# For example, 'Pavia, Italy' returns the comune of Pavia rather than the city of Pavia.
# In those cases, it is better to include a zip code, e.g., 'Pavia, 27100, Italy'
city = "Verona, Italy"

# Arrange geolocator
geolocator = Nominatim(user_agent="30daymapchallenge", timeout=100)

# Attempt to query location based on city name
location = geolocator.geocode(city)

# Check is successful
city_lng, city_lat = None, None
if location:
    city_lng, city_lat = location.longitude, location.latitude
    print(f"Successful geolocation for: {city} - ({city_lng}, {city_lat})")
else:
    print(f"Failed geolocation for: {city}")

Successful geolocation for: Verona, Italy - (10.9924122, 45.4384958)


In [4]:
# Configure Overpass API
# NOTE: There are some limited requests per length of time on this API
endpoint = "https://overpass-api.de/api/interpreter"
api = overpass.API(timeout=100, endpoint=endpoint)

# Create bounding box with a buffer around city center point
buffer_lat, buffer_lng = 0.01, 0.02
bbox = [city_lat-buffer_lat, city_lng-buffer_lng, city_lat+buffer_lat, city_lng+buffer_lng]

# Arrange query
bbox_str = ",".join([str(b) for b in bbox])
query = '''way["building"](''' + bbox_str + '''); out geom;'''

# Get response
result = api.get(query, responseformat="geojson")

# Arrange into dataframe
df = pd.DataFrame(result)

# Explode features into columns
df = pd.concat([df, df['features'].apply(pd.Series)], axis=1)

# Parse geometry and save as geodataframe
gdf = gpd.GeoDataFrame(df.drop(columns="geometry"), geometry=df.geometry.apply(shape))

# Preview
print(f"Found {gdf.shape[0]} building footprints")
gdf.head(5)

Found 16440 building footprints


,type,features,type,id,properties,geometry
0,FeatureCollection,"{'type': 'Feature', 'id': 71748433, 'geometry'...",Feature,71748433,{'building': 'yes'},"LINESTRING (11.01096 45.43851, 11.01119 45.438..."
1,FeatureCollection,"{'type': 'Feature', 'id': 71748463, 'geometry'...",Feature,71748463,{'building': 'yes'},"LINESTRING (11.00361 45.43639, 11.00360 45.436..."
2,FeatureCollection,"{'type': 'Feature', 'id': 71748572, 'geometry'...",Feature,71748572,"{'building': 'yes', 'name': 'Polo Zanotto'}","LINESTRING (11.00408 45.43636, 11.00404 45.436..."
3,FeatureCollection,"{'type': 'Feature', 'id': 116634447, 'geometry...",Feature,116634447,"{'alt_name': 'Ponte di Castelvecchio', 'bridge...","LINESTRING (10.98708 45.44103, 10.98708 45.440..."
4,FeatureCollection,"{'type': 'Feature', 'id': 127708649, 'geometry...",Feature,127708649,{'building': 'yes'},"LINESTRING (11.00557 45.43070, 11.00577 45.430..."


In [5]:
# Construct plot
fig, ax = plt.subplots(figsize=(10,6))
gdf.geometry.plot(ax=ax, facecolor="DarkSlateGray", edgecolor="w", linewidth=0.05)
ax.axis('off')
ax.set_title(city, fontsize="xx-large", color="DarkSlateGray") 
plt.tight_layout()

# Write to path
out_path = os.path.join("..", "contributions", f"03_{city.split(',').pop(0)}")
fig.savefig(out_path, facecolor="w", dpi=300)

# Preview
plt.show()